In [1]:
import attk
import os
import numpy as np
import librosa
from IPython.display import display, Audio
import timeit
import random
from itertools import groupby
from operator import itemgetter
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
import subprocess
import unicodecsv
import urllib2

os.chdir('/sharedfolder/GitHub/sida/')

!mkdir /sharedfolder/GitHub/sida/___training_audio

os.chdir('/sharedfolder/GitHub/sida/___training_audio')


mkdir: cannot create directory '/sharedfolder/GitHub/sida/___training_audio': File exists


In [2]:
## Loading label data CSV as a list of lists

csv_url = "https://raw.githubusercontent.com/hipstas/aapb-labels/master/Universal_Background_Model_AAPB_4K_Corpus/aapb_ubm_4K_2017.csv"

csv_string = urllib2.urlopen(csv_url)

train_table = []

csv_reader = unicodecsv.reader(csv_string)

for row in csv_reader:
        train_table.append(row)

train_table[:5]+['...']

[[u'Media file basename', u'Start time', u'Duration', u'Label', u'Labeled by'],
 [u'cpb-aacip-55-19s1sc16.h264__2145.0_2155.0',
  u'2',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-29-6663z15p.h264__121.0_131.0',
  u'3',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-235-12z35scw.h264__212.0_222.0',
  u'3',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-500-7s7hv789__149.0_159.0',
  u'2',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 '...']

In [3]:
## Removing header row if present

if 'Media file basename' in train_table[0]:
    train_table = train_table[1:]

random.shuffle(train_table)

In [ ]:
%%capture
## Excerpting labeled WAV clips

training_audio_pathname = "/sharedfolder/audio_labeler/media/ubm_clips_final_July_2017/"
out_dir = '_classes_' + training_audio_pathname.strip('/').split('/')[-1]


for row in train_table:
    try:
        basename , start, duration, class_name, labeled_by = row  ## Assigning values in row to variables
        filename = str(basename + '.wav')
        start = float(start)
        end = float(start) + float(duration)
        out_pathname = str(os.path.join(out_dir, class_name.replace(' ','_')))
        try: 
            subprocess.call(['mkdir', '-p', out_pathname])
        except:
            pass
        attk.subclip(os.path.join(training_audio_pathname, filename), float(start), end, out_pathname) ## <- attk
    except Exception as e: 
        print(row)
        print(e)



In [ ]:
os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017')

In [ ]:
for class_dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    batch_extract_vowels(class_dir_name)

In [3]:
import csv

## Extracting features
tic=timeit.default_timer()

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017')


dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    
    os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017/' + dir_name + '/_vowel_clips')
    
    try: os.mkdir('../_vowel_mfccs_and_deltas')
    except: pass
    
    for filename in [item for item in os.listdir('./') if '.wav' in item.lower()]:
        mfccs = attk.get_mfccs_and_deltas(filename)
        with open('../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv', 'w') as fo:
            csv_writer = csv.writer(fo)
            csv_writer.writerows(mfccs)  


print(timeit.default_timer() - tic)

tic=timeit.default_timer()

##


462.754571915


NameError: name 'speaker_1_mfccs' is not defined